Some of the imports are not useful yet

In [5]:
from __future__ import print_function
import os
import matplotlib.pyplot as plt
import numpy as np
import theano
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from yaafelib import *

Download GTZAN dataset once

In [ ]:
def downloadGTZAN():
    filename = 'genres.tar.gz'
    if not os.path.exists(filename):
        filename, _ = urlretrieve('http://opihi.cs.uvic.ca/sound/genres.tar.gz', filename)
    else:
        print('File ' + filename + ' exists')
    
    return filename
filename = downloadGTZAN()

Extract files from .tar.gz

In [14]:
num_classes = 10

def extract(filename):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root):
        print('File %s already extracted' % (root))
    else:
        print('Extracting %s ' % filename)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))
                    if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
        raise Exception('Expected %d folders not found.' % (num_classes))
    print(data_folders)
    return data_folders

folders = extract(filename)

Extracting genres.tar.gz 
['genres/blues', 'genres/classical', 'genres/country', 'genres/disco', 'genres/hiphop', 'genres/jazz', 'genres/metal', 'genres/pop', 'genres/reggae', 'genres/rock']


In [ ]:
def get_features(audio_file):
    if os.path.exists(audio_file):
        print('Getting features from ' + audio_file)
    else:
        raise Exception('File ' + audio_file + ' not found')
    fp = FeaturePlan(sample_rate=22050, normalize=1)
    # Features that seems to be most often used, so they are good to start with.
    fp.addFeature('mfcc: MFCC')
    fp.addFeature('zcr: ZCR')
    fp.addFeature('spectral_shape: SpectralShapeStatistics')
    fp.addFeature('magnitude_spectrum: MagnitudeSpectrum')
    df = fp.getDataFlow()
    engine = Engine()
    engine.load(df)
    afp = AudioFileProcessor()
    afp.setOutputFormat('csv', 'features', {'Precision': '8', 'Metadata': 'False'})
    afp.processFile(engine, audio_file)
    engine.flush()
    feats = engine.readAllOutputs()
    return feats

def load_genre(folder):
    print('Loading genre from folder ' + folder)
    samples = os.listdir(folder)
    dataset =[]
    for sample in os.listdir(folder):
        sample_file = os.path.join(folder, sample)
        if sample.endswith('.au'):
            features = get_features(sample_file)
            dataset.append(features)
    return dataset

def pickle(data_folders):
    dataset_names = []
    for folder in data_folders:
        set_filename = folder + '.pickle'
        dataset_names.append(set_filename)
        if os.path.exists(set_filename):
            print('%s already pickled' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_genre(folder)
            with open(set_filename, 'wb') as f:
                pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
    return dataset_names

pickled_datasets = pickle(folders)

Now the data is pickled, thus the feature extraction doesn't have to be repeated. 

Let's vizualize something

In [ ]:
%matplotlib inline  

def visualize(filename, sample, feature):
    with open(filename, 'r') as f:
        unpickled = pickle.load(f)
        f = unpickled[sample][feature]
        plt.plot(f[0])
        plt.ylabel(feature)
        plt.show()

visualize('genres/rock.pickle', 0, 'magnitude_spectrum')
visualize('genres/rock.pickle', 1, 'magnitude_spectrum')
visualize('genres/rock.pickle', 2, 'magnitude_spectrum')
visualize('genres/rock.pickle', 3, 'magnitude_spectrum')
visualize('genres/blues.pickle', 0, 'magnitude_spectrum')
visualize('genres/blues.pickle', 1, 'magnitude_spectrum')
visualize('genres/blues.pickle', 2, 'magnitude_spectrum')
visualize('genres/blues.pickle', 3, 'magnitude_spectrum')

Training set and test set are needed. In addition, they have to be shuffeled/randomized somehow.

In [ ]:
pickled_datasets = ['blues.pickle','classical.pickle','country.pickle','disco.pickle','hiphop.pickle','jazz.pickle',
                    'metal.pickle','pop.pickle','reggae.pickle','rock.pickle']

def unpickle(filename):
    with open('genres/' + filename, 'r') as f:
        return pickle.load(f)
    
def evens(dataset):
    return dataset[::2]
def odds(dataset):
    return dataset[1::2]

def dataset_with_labels(filename):
    dataset = unpickle(filename)
    return dataset, [os.path.splitext(filename)[0] for count in xrange(len(dataset))]

datasets = map(dataset_with_labels, pickled_datasets)
features = [features for genre in datasets for features in genre[0]]
labels = [label for genre in datasets for label in genre[1]]

train_dataset = odds(features)
train_labels = odds(labels)
test_dataset = evens(features)
test_labels = evens(labels)

def randomize(dataset, labels):
    permutation = np.random.permutation(len(labels))
    shuffled_dataset = np.asarray(dataset)[permutation]
    shuffled_labels = np.asarray(labels)[permutation]
    return shuffled_dataset, shuffled_labels

train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

try:
    f = open('train_and_test_data.pickle', 'wb')
    save = {
        'train_features': train_dataset,
        'train_labels': train_labels,
        'test_features': test_dataset,
        'test_labels': test_labels,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

Now, we can go to TensorFlow

You can start here
=====

In [4]:
with open('train_and_test_data.pickle', 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_features']
    train_labels = save['train_labels']
    test_dataset = save['test_features']
    test_labels = save['test_labels']
    del save  # gc
train_dataset = ([d['magnitude_spectrum'][0] for d in train_dataset])
test_dataset = ([d['magnitude_spectrum'][0] for d in test_dataset])
magnitude = 513
num_labels = 10 # genres

import numpy as np

def oneHotEncoder(pos, max):
    encoded = []
    for i in range(0, max):
        if i == pos:
            encoded.append(1)
        else:
            encoded.append(0)
    return encoded

def reformat(dataset, labels):
    dataset = np.asarray(dataset).reshape((-1, magnitude, 1, 1)).astype(np.float32)
    genres = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
    labels = map(lambda x: np.int32(genres.index(x)), labels)
    labels = map(lambda x: oneHotEncoder(x, num_labels), labels)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

In [12]:
import theano.tensor as T

rng = np.random
floatX = theano.config.floatX

class Classifier(object):
    def __init__(self, n_features):
        hidden_layer_size = 5
        l2_regularisation = 0.001

        input_vector = T.fvector('input_vector')
        target_value = T.fscalar('target_value')
        learningrate = T.fscalar('learningrate')

        W_hidden_vals = np.asarray(rng.normal(loc=0.0,
                                                 scale=0.1,
                                                 size=(n_features, hidden_layer_size)),
                                      dtype=floatX)
        W_hidden = theano.shared(W_hidden_vals, 'W_hidden')

        hidden = T.dot(input_vector, W_hidden)
        hidden = T.nnet.sigmoid(hidden)

        W_output_vals = np.asarray(rng.normal(loc=0.0,
                                                 scale=0.1,
                                                 size=(hidden_layer_size, 1)),
                                      dtype=floatX)
        W_output = theano.shared(W_output_vals, 'W_output')

        predicted_value = T.dot(hidden, W_output)
        predicted_value = T.nnet.sigmoid(predicted_value)

        cost = T.sqr(predicted_value - target_value).sum()
        cost += l2_regularisation * (T.sqr(W_hidden).sum() + T.sqr(W_output).sum())

        params = [W_hidden, W_output]
        gradients = T.grad(cost, params)
        updates = [(p, p - (learningrate * g)) for p, g in zip(params, gradients)]

        self.train = theano.function(inputs=[input_vector, target_value, learningrate],
                                     outputs=[cost, predicted_value],
                                     updates=updates,
                                     allow_input_downcast=True)

        self.test = theano.function(inputs=[input_vector, target_value],
                                    outputs=[cost, predicted_value],
                                    allow_input_downcast=True)


def create_random_dataset(num_of_classes, num_of_vectors, num_of_features, feature_min_value, features_max_value):
    dataset = []
    for i in range(num_of_vectors):
        classify_as = rng.randint(1, num_of_classes)
        features_vector = rng.uniform(feature_min_value, features_max_value, num_of_features)
        dataset.append((classify_as, features_vector))
    return dataset


def create_classes_ranges(num_of_classes, min_value, max_value):
    weights_ranges = [min_value]
    diff = (max_value - min_value) / num_of_classes
    previous = min_value
    for i in range(1, num_of_classes):
        previous += diff
        weights_ranges.append(previous)
    weights_ranges.append(max_value)
    return weights_ranges


learningrate = 0.1
epochs = 25

num_classes = 4
num_features = 4
data_train_size = 100
data_test_size = 20

data_train = create_random_dataset(num_classes, data_train_size, num_features, -1.0, 1.0)
data_test = create_random_dataset(num_classes, data_test_size, num_features, -1.0, 1.0)

weights_values = create_classes_ranges(num_classes, 0.0, 1.0)
print(weights_values)

classifier = Classifier(num_features)

[0.0, 0.25, 0.5, 0.75, 1.0]


There is a huge field for improvements here:
* Starting points should not be randomized

Training & Testing:
=========================

In [13]:
print("Neural network test...")
print("Number of classes: ", num_classes)
print("Number of features: ", num_features)
print("Epochs: ", epochs)
print("Training data:")
print(data_train)
print("Testing data:")
print(data_test)

for epoch in range(epochs):
    cost_sum = 0.0
    correct = 0
    for label, vector in data_train:
        cost, predicted_value = classifier.train(vector, label, learningrate)
        cost_sum += cost
        range_index = int(label)
        if (predicted_value >= weights_values[range_index]) and (predicted_value < weights_values[range_index + 1]):
            correct += 1
    print("Epoch: " + str(epoch) + " Cost: " + str(cost_sum), ", Accuracy: " + str(float(correct) / len(data_train)))

cost_sum = 0.0
correct = 0
index = 0
for label, vector in data_test:
    index += 1
    cost, predicted_value = classifier.test(vector, label)
    cost_sum += cost
    range_index = int(label)
    if (predicted_value >= weights_values[range_index]) and (predicted_value < weights_values[range_index + 1]):
        correct += 1
    print("Test_cost: " + str(cost_sum) + ", Test_accuracy: " + str(float(correct) / index))
print("Number of correct guesses: ", correct, " out of ", data_test_size)

Neural network test...
Number of classes:  4
Number of features:  4
Epochs:  25
Training data:
[(1, array([ 0.18859198,  0.83224269,  0.23613818, -0.91101835])), (2, array([-0.22595652, -0.32682014, -0.63354295,  0.04799655])), (1, array([-0.40658881,  0.05777961, -0.57866686, -0.93151891])), (3, array([ 0.54505641, -0.95898673, -0.30147757,  0.38201886])), (3, array([ 0.99996535,  0.28485004,  0.26605391,  0.72069321])), (1, array([-0.95753242, -0.0036907 ,  0.72367905,  0.02785985])), (1, array([-0.93181203, -0.85121938,  0.06947845, -0.38879619])), (3, array([ 0.80214545,  0.25432567, -0.15890534,  0.16948516])), (3, array([-0.8441584 , -0.55379886, -0.27441706, -0.56673955])), (2, array([-0.40939481,  0.08088165, -0.87541221, -0.02193611])), (2, array([ 0.87229882,  0.0606048 , -0.55171368, -0.87291553])), (3, array([-0.53220096,  0.62423546, -0.14385187, -0.32438767])), (1, array([ 0.99243767,  0.99014379,  0.57228016,  0.69527707])), (3, array([ 0.81119975, -0.87311573, -0.305972

Classifier is disastrous. Has to be enhanced, but it's somehow working...